# SPCS Azure Managed Instance Connectivity Test

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

This notebook provides modular components for testing Azure SQL Managed Instance connectivity from Snowflake Container Services (SPCS):

## Quick Start Options:
1. **Basic Test**: Configure parameters and run network/auth tests
2. **Full Setup**: Use cells to set up PyPI access and SQL Managed Instance EAI as needed
3. **Custom Setup**: Pick and choose individual components as needed

## Components Available:
- **Configuration**: Set your SQL Managed Instance connection parameters
- **PyPI Setup**: Optional - enables installing pymssql driver from PyPI
- **Network Test**: Tests basic network connectivity to Managed Instance host
- **Authentication Test**: Tests Managed Instance authentication and basic queries using pymssql
- **Managed Instance EAI**: Optional - configures External Access Integration for your Azure SQL Managed Instance


**Note: You must restart the Session after making EAI changes for them to take effect**

In [ ]:
# Managed Instance Connectivity Test Configuration
# Update these parameters with your actual Managed Instance connection details

# User Configuration - UPDATE THIS
SQL_INSTANCE_HOST = "snow-sql-azure.3b55c0b4fb11.database.windows.net"
SQL_INSTANCE_PORT = 1433
SQL_INSTANCE_DATABASE = "snowdb"
SQL_INSTANCE_USER = "adminsql"
SQL_INSTANCE_PASSWORD = "jeg1jzt4hja-vhm4RCF"

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# This role will be used by Openflow to run Connectors and perform your Data Engineering tasks
OPENFLOW_RUNTIME_ROLE = "ACCOUNTADMIN"

print(f"Configuration:")
print(f"SQL Managed Instance Host: {SQL_INSTANCE_HOST}")
print(f"SQL Managed Instance Port: {SQL_INSTANCE_PORT}")
print(f"SQL Managed Instance Database: {SQL_INSTANCE_DATABASE}")
print(f"SQL Managed Instance User: {SQL_INSTANCE_USER}")
print("\nReady to test connectivity...")

## 1. PyPI Setup (Optional)

Run these cells if you need to install the pymssql driver from PyPI. This creates the necessary network rules and External Access Integration for PyPI access.

**Skip this section if you already have pymssql installed or have PyPI access configured.**


In [ ]:
-- Create Network Rule and External Access Integration for PyPI
-- Run this cell to enable installing Python packages from PyPI

USE ROLE {{IMPLEMENTATION_ROLE}};

CREATE OR REPLACE NETWORK RULE pypi_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('pypi.org', 'pypi.python.org', 'pythonhosted.org', 'files.pythonhosted.org');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION pypi_access_integration
  ALLOWED_NETWORK_RULES = (pypi_network_rule)
  ENABLED = true
  COMMENT = 'External Access Integration for PyPI package installation';

-- Grant usage on the integration
GRANT USAGE ON INTEGRATION pypi_access_integration TO ROLE {{IMPLEMENTATION_ROLE}};

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'pypi_access_integration';


In [ ]:
-- Apply PyPI integration to this notebook
-- Run this after creating the PyPI integration above

ALTER NOTEBOOK EAI_MANAGED_SQL
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('pypi_access_integration', 'MANAGED_SQL_EAI');

  -- Restart your Notebook session after applying an EAI


**NOTE: You will need to restart the Notebook session after applying an EAI**

In [ ]:
# Install MSSQL driver (pymssql)
# Make sure PyPI access is configured first if you get connection errors
# You can run this cell twice; the first to install the driver, the second to confirm it is imported

try:
    import pymssql
    print("✅ pymssql already available")
except ImportError:
    print("📦 Installing pymssql...")
    %pip install pymssql
    print("✅ pymssql installed")


## 2. Connectivity Tests

These cells test connectivity to your Managed Instance. Run them in order to diagnose any connection issues.

**Note: If you need to install pymssql, make sure to run the PyPI setup section first and restart your session.**


In [ ]:
# Network Connectivity Test
# Tests basic network connectivity to Managed Instance SQL host and port

import socket

def test_network_connectivity(host, port, description):
    """Test network connectivity to Managed Instance host and port"""
    try:
        print(f"🔍 Testing {description}: {host}:{port}")

        # Test network connectivity
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(10)
        result = sock.connect_ex((host, port))
        sock.close()

        if result == 0:
            print(f"  ✅ Network connection successful")
            return True
        else:
            print(f"  ❌ Network connection failed (error code: {result})")
            print(f"  💡 This may indicate the need for a Managed Instance EAI")
            return False

    except Exception as e:
        print(f"  ❌ Network Error: {e}")
        return False

# Run network connectivity test
print("=" * 50)
print("NETWORK CONNECTIVITY TEST")
print("=" * 50)

network_result = test_network_connectivity(SQL_INSTANCE_HOST, SQL_INSTANCE_PORT, "Managed Instance Network Connectivity")

if network_result:
    print(f"\n✅ Network connectivity PASSED - Managed Instance host is reachable")
    print("You can proceed to test Managed Instance DB authentication.")
else:
    print(f"\n❌ Network connectivity FAILED")
    print("You need to configure a Managed Instance External Access Integration (EAI).")
    print("See the Managed Instance EAI Setup section below.")

In [ ]:
# SQL Server (Azure SQL Managed Instance) Authentication Test
# Using pure-Python pymssql (works in SPCS)

import pymssql

def test_sqlmi_authentication():
    """Test SQL Managed Instance authentication and basic query"""
    try:
        print(f"🔍 Testing SQL Managed Instance Authentication and Basic Query")

        print("  📦 Using pymssql library (pure Python)")

        # Create connection
        conn = pymssql.connect(
            server=SQL_INSTANCE_HOST,
            user=SQL_INSTANCE_USER,
            password=SQL_INSTANCE_PASSWORD,
            database=SQL_INSTANCE_DATABASE,
            port=int(SQL_INSTANCE_PORT),
            tds_version='7.4',     # Important for Azure SQL / TLS 1.2
            login_timeout=15,
            timeout=30,
            charset='UTF-8'
        )

        print(f"  ✅ Authentication successful")

        # Test basic query
        cursor = conn.cursor()
        cursor.execute("SELECT @@VERSION;")
        version = cursor.fetchone()[0]
        print(f"  ✅ Database query successful")
        print(f"  📊 SQL Server Version: {version.splitlines()[0]}")

        # List top 5 tables
        cursor.execute("""
            SELECT TOP 5 TABLE_SCHEMA, TABLE_NAME
            FROM INFORMATION_SCHEMA.TABLES
            ORDER BY TABLE_SCHEMA, TABLE_NAME;
        """)
        tables = cursor.fetchall()
        print(f"  📋 Found {len(tables)} tables")
        for schema, table in tables:
            print(f"     - {schema}.{table}")

        cursor.close()
        conn.close()
        return True

    except Exception as e:
        print(f"  ❌ Database Error: {e}")
        print(f"  💡 Check credentials, firewall rules, or network connectivity")
        return False


# Example usage
if __name__ == "__main__":
    print("=" * 50)
    print("AZURE SQL MANAGED INSTANCE AUTHENTICATION TEST")
    print("=" * 50)

    auth_result = test_sqlmi_authentication()

    if auth_result:
        print(f"\n✅ SQL Managed Instance authentication PASSED")
        print("Your environment can successfully connect to Azure SQL MI!")
    else:
        print(f"\n❌ SQL Managed Instance authentication FAILED")
        print("Check your credentials or network connectivity.")


## 3. SQL Managed Instance EAI Setup (Optional)

Run these cells if connectivity tests failed. This creates the necessary network rules and External Access Integration specifically for Managed Instance database access.

**Skip this section if your connectivity tests passed.**

In [ ]:
-- Create Network Rule for Managed Instance connectivity
-- This allows egress traffic to your specific Managed Instance host and port

USE ROLE {{IMPLEMENTATION_ROLE}};

-- Network rule for your Managed Instance database server
-- Uses the variables set in the configuration cell above
CREATE OR REPLACE NETWORK RULE MANAGED_SQL_RULE
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('{{SQL_INSTANCE_HOST}}:{{SQL_INSTANCE_PORT}}');

-- Show the created network rule
DESCRIBE NETWORK RULE MANAGED_SQL_RULE;

In [ ]:
-- Create External Access Integration for Managed Instance
-- This combines all the network rules into a single integration

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION MANAGED_SQL_EAI
  ALLOWED_NETWORK_RULES = (
    MANAGED_SQL_RULE
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for Managed Instance connectivity';

-- Grant usage on the integration to your role
GRANT USAGE ON INTEGRATION MANAGED_SQL_EAI TO ROLE {{IMPLEMENTATION_ROLE}};
GRANT USAGE ON INTEGRATION MANAGED_SQL_EAI TO ROLE {{OPENFLOW_RUNTIME_ROLE}};

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'MANAGED_SQL_EAI';

In [ ]:
-- Apply Managed Instance EAI to this notebook
-- This enables the notebook to access your Managed Instance database
-- Include pypi_access_integration if you created it earlier

ALTER NOTEBOOK EAI_MANAGED_SQL
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('MANAGED_SQL_EAI', 'pypi_access_integration');

  -- Restart your Notebook session after applying an EAI